# AI Broschure Generator
### Buiseness Challenge:
Create a product that builds a Broschure for a company to be used for prospective clients, investors and potential recruiters.

We will be provided the company name and their primary website.

In [4]:
import os 
import requests 
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
load_dotenv()
google_api_key = os.getenv('GEMINI_API_KEY')

model = "gemini-1.5-flash"
gemini = ChatGoogleGenerativeAI(
    model = model,
    temperature=0.4,
    google_api_key=google_api_key
)


In [ ]:
# A class to represent a webpa